### Importing Libraries

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("PyTorch version:", torch.__version__)
if device.type == 'cuda':
    print("CUDA version:", torch.version.cuda)
    print("Device name:", torch.cuda.get_device_name(0))

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

Using device: cuda
PyTorch version: 2.8.0+cu129
CUDA version: 12.9
Device name: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import pandas as pd
from tqdm import tqdm
import ollama

tqdm.pandas()  # enable progress bar

In [3]:
def rectify_message(row):
    file_name = str(row.get("File Name", "file"))
    dev_msg = str(row.get("Message", ""))
    llm_msg = str(row.get("LLM Inference", ""))
    diff = str(row.get("Diff", ""))
    change_type = str(row.get("Change Type", ""))

    prompt = f"""
You are refining commit messages for bug-fixing commits.

File: {file_name}
Change type: {change_type}

Git Diff:
{diff}

Developer's commit message: {dev_msg if dev_msg else "N/A"}
LLM inference: {llm_msg if llm_msg else "N/A"}

Task:
Generate a **detailed rectified commit message** in this style:
[{file_name}]: Fix [bug/issue/etc] in [function/component] by [specific action].
Keep under 20 words, precise, and avoid vague terms like 'update' or 'change'.
"""

    try:
        response = ollama.chat(model="codellama:7b", messages=[
            {"role": "user", "content": prompt}
        ])
        rectified = response["message"]["content"].strip()
        return rectified if rectified else f"{file_name}: Minor {change_type}"
    except Exception as e:
        return f"{file_name}: Minor update"

In [4]:
try:
    df = pd.read_csv("results/diffs_per_file_with_llm_infer.csv")
    print("Starting commit message rectification...")
    # df = df.head(10).copy()  # For testing, remove or adjust as needed
    # df["Rectified Message"] = df.progress_apply(rectify_message, axis=1)
    rectified_messages = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Rectifying messages..."):
        rectified_messages.append(rectify_message(row))
    df["Rectified Message"] = rectified_messages

    output_filename = "results/ollama_rectified.csv"
    df.to_csv(output_filename, index=False)

    print(f"Rectification complete. Output saved to {output_filename}")

except Exception as e:
    print("Error:", e)

Starting commit message rectification...


Rectifying messages...: 100%|██████████| 2041/2041 [38:15<00:00,  1.12s/it] 


Rectification complete. Output saved to results/ollama_rectified.csv


In [ ]:
df = pd.read_csv("results/ollama_rectified_commits.csv")
display(df.head(20))

,Hash,Message,File Name,File Path,Change Type,Source Code (before),Source Code (current),Diff,LLM Inference (fix type),Rectified Message
0,014a277a97759bbc0e6ec8fba588bc6e6de65a86,A few fixes to initial point_thickness impleme...,constants.py,constants.py,ModificationType.MODIFY,import os\nimport numpy as np\n\n\nGENERALLY_B...,import os\nimport numpy as np\n\nPRODUCTION_QU...,"@@ -1,9 +1,6 @@\n import os\n import numpy as ...",add missing constants,[constants.py]: Fix default point thickness to...
1,014a277a97759bbc0e6ec8fba588bc6e6de65a86,A few fixes to initial point_thickness impleme...,displayer.py,displayer.py,ModificationType.MODIFY,import numpy as np\nimport itertools as it\nim...,import numpy as np\nimport itertools as it\nim...,"@@ -55,7 +55,7 @@ def paint_mobjects(mobjects,...",add nudge to displayer.py,[displayer.py]: Fix potential offset issue in ...
2,014a277a97759bbc0e6ec8fba588bc6e6de65a86,A few fixes to initial point_thickness impleme...,mobject.py,mobject\mobject.py,ModificationType.MODIFY,import numpy as np\nimport itertools as it\nim...,import numpy as np\nimport itertools as it\nim...,"@@ -21,7 +21,7 @@ class Mobject(object):\n ...",add missing docstring,[mobject.py]: Fix inconsistent point_thickness...
3,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",__init__.py,__init__.py,ModificationType.MODIFY,from animation import *\nfrom mobject import *...,from animation import *\nfrom scene import *\n...,"@@ -1,10 +1,13 @@\n from animation import *\n-...",add missing import statements,[__init__.py]: Fix [bug/issue/etc] in [functio...
4,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",__init__.py,animation\__init__.py,ModificationType.MODIFY,from animation import *\nfrom transform import...,from animation import *\nfrom meta_animations ...,"@@ -1,3 +1,4 @@\n from animation import *\n-fr...",add missing newline,Fix regression in transform module by adding m...
5,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",animation.py,animation\animation.py,ModificationType.MODIFY,from PIL import Image\nfrom colour import Colo...,from PIL import Image\nfrom colour import Colo...,"@@ -7,11 +7,10 @@ import os\n import copy\n im...",add missing config to missing color animation,[animation.py]: Fix bug in writeGif function b...
6,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",meta_animations.py,animation\meta_animations.py,ModificationType.ADD,NaN,import numpy as np\nimport itertools as it\nfr...,"@@ -0,0 +1,95 @@\n+import numpy as np\n+import...",add tests for animation.update and animation.u...,[meta_animations.py]: Fix bug in DelayByOrder ...
7,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",simple_animations.py,animation\simple_animations.py,ModificationType.MODIFY,import numpy as np\nimport itertools as it\nfr...,import numpy as np\nimport itertools as it\nfr...,"@@ -2,42 +2,12 @@ import numpy as np\n import ...",add some more classes to the animation class,[simple_animations.py]: Refactor commit messag...
8,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",transform.py,animation\transform.py,ModificationType.MODIFY,import numpy as np\nimport itertools as it\nim...,import numpy as np\nimport itertools as it\nim...,"@@ -4,39 +4,12 @@ import inspect\n import copy...",add new animation classes,[transform.py]: Fix broken behavior in path_al...
9,2e074afb60d13262ce1e42e83bcf0ed28d95ad82,"middle of massive restructure, everything stil...",displayer.py,displayer.py,ModificationType.MODIFY,import numpy as np\nimport itertools as it\nim...,import numpy as np\nimport itertools as it\nim...,"@@ -8,8 +8,7 @@ import cv2\n from colour impor...",add more examples to the nightly modified file,[displayer.py]: Fix compression issue in write...
